In [148]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import os
#import requests
import pandas as pd
from skimage import morphology
from skimage.color import rgb2hsv
from scipy.spatial.distance import cdist
from scipy.stats.stats import mode
from PIL import Image
from collections import Counter
import cv2
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
import glob
import random
from sklearn.model_selection import train_test_split
#import seaborn as sns

import concurrent.futures

In [149]:
# from sklearn.datasets import load_files

image_folder_path = "/Users/theakaroline/Documents/GitHub/Cherry2/processed_images"
#This folder contains the 100 masked sample images
#this should be only the traing data


#here the ground Truth shoud be as well.
ground_truth = '/Users/theakaroline/Documents/GitHub/Cherry2/ground_truth.csv'
# my_data = load_files(data_folder_path, categories =['folder1', 'folder2', 'folder3'])


In [229]:
images = os.listdir(image_folder_path)

groundTruth_df = pd.read_csv(ground_truth, delimiter = ";")

groundTruth_df = groundTruth_df.set_index("ID")


groundTruth_df['Cancerous'] = ((groundTruth_df['BCC'] | groundTruth_df['MEL'] | groundTruth_df['SCC']) == 1).astype(int)
groundTruth_df
#groundTruth = groundTruth_df.set_index("ID")
#this line chooses the photo ID to be the index

,BCC,MEL,SCC,ACK,NEV,SEK,Cancerous
ID,,,,,,,
PAT_1022_115_132,0,0,0,1,0,0,0
PAT_1026_124_346,0,0,0,1,0,0,0
PAT_1027_128_854,0,0,0,1,0,0,0
PAT_1129_498_930,0,0,0,0,1,0,0
PAT_1170_609_378,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...
PAT_944_1795_666,1,0,0,0,0,0,1
PAT_952_1807_58,1,0,0,0,0,0,1
PAT_97_151_503,1,0,0,0,0,0,1


In [235]:
to_be_cancerous_or_not_to_be = groundTruth_df.loc[:, ['Cancerous']]
#to_be_cancerous_or_not_to_be = groundTruth_df[['ID', 'Cancerous']].copy()
to_be_cancerous_or_not_to_be

,Cancerous
ID,
PAT_1022_115_132,0
PAT_1026_124_346,0
PAT_1027_128_854,0
PAT_1129_498_930,0
PAT_1170_609_378,0
...,...
PAT_944_1795_666,1
PAT_952_1807_58,1
PAT_97_151_503,1


In [153]:
the_path = groundTruth_df.index.tolist()



['PAT_1022_115_132',
 'PAT_1026_124_346',
 'PAT_1027_128_854',
 'PAT_1129_498_930',
 'PAT_1170_609_378',
 'PAT_1174_628_915',
 'PAT_1203_724_762',
 'PAT_1304_3507_308',
 'PAT_1306_1086_124',
 'PAT_1315_1110_317',
 'PAT_1414_1432_969',
 'PAT_1414_1433_570',
 'PAT_1441_1531_670',
 'PAT_1460_1598_746',
 'PAT_1475_1651_727',
 'PAT_1492_1705_578',
 'PAT_1571_2484_488',
 'PAT_1575_2516_23',
 'PAT_1754_3308_390',
 'PAT_1764_3335_803',
 'PAT_1810_3484_347',
 'PAT_1811_3487_560',
 'PAT_1821_3575_920',
 'PAT_1874_3687_799',
 'PAT_1914_3822_863',
 'PAT_1922_3848_451',
 'PAT_1942_3917_379',
 'PAT_1946_3924_627',
 'PAT_1995_4080_695',
 'PAT_2034_4269_670',
 'PAT_2062_4397_39',
 'PAT_2076_4459_450',
 'PAT_2105_4587_204',
 'PAT_2124_4650_575',
 'PAT_2127_4662_596',
 'PAT_228_348_2',
 'PAT_235_360_50',
 'PAT_245_376_24',
 'PAT_247_378_484',
 'PAT_248_379_695',
 'PAT_279_1393_118',
 'PAT_288_441_312',
 'PAT_301_649_533',
 'PAT_304_653_446',
 'PAT_306_1646_971',
 'PAT_313_669_935',
 'PAT_330_1437_458',


In [242]:
#We need some sort of code to classify the pictures

cancerous_mask = (groundTruth_df["BCC"] == 1) + (groundTruth_df["MEL"] == 1) + (groundTruth_df["SCC"] == 1)
non_cancerous_mask = (groundTruth_df["ACK"] == 1) + (groundTruth_df["NEV"] == 1) + (groundTruth_df["SEK"] == 1)
print(len(cancerous_mask))
print(len(non_cancerous_mask))

cancerous_mask = pd.DataFrame(cancerous_mask, columns = ['Value'])
non_cancerous_mask = pd.DataFrame(non_cancerous_mask, columns = ['Value'])


cancerous_mask = cancerous_mask.loc[cancerous_mask.Value, :]
non_cancerous_mask = non_cancerous_mask.loc[non_cancerous_mask.Value, :]
print(len(cancerous_mask))
print(len(non_cancerous_mask))
cancerous_mask_list = cancerous_mask.index.tolist()
non_cancerous_mask_list = non_cancerous_mask.index.tolist()

cancerous_mask["Value"] = cancerous_mask["Value"].astype(int)
non_cancerous_mask["Value"] = non_cancerous_mask["Value"].astype(int)
non_cancerous_mask=non_cancerous_mask.replace(1,0)
print(len(cancerous_mask))
print(len(non_cancerous_mask))


98
98
47
51
47
51


In [218]:
#cancerous_images = groundTruth_df.index[cancerous_mask].tolist()
#non_cancerous_images = groundTruth_df.index[non_cancerous_mask].tolist()
# # #So, cancerous_images is a list of image IDs for cancerous skin lesions 
# # in our GroundTruth dataframe

path_to_all_images = []
for i in the_path:
     path_to_all = os.path.join(image_folder_path, i + ".png")
     path_to_all_images.append(path_to_all)

# #path to cancerous images
cancerous_with_masks = []
for i in cancerous_mask_list:
     cancerous_path = os.path.join(image_folder_path, i + ".png")
     cancerous_with_masks.append(cancerous_path)

# # #path to non cancerous images
non_cancerous_with_masks = []
for i in non_cancerous_mask_list:
     non_cancerous_path = os.path.join(image_folder_path, i + ".png")
     non_cancerous_with_masks.append(non_cancerous_path)



['/Users/theakaroline/Documents/GitHub/Cherry2/processed_images/PAT_1022_115_132.png',
 '/Users/theakaroline/Documents/GitHub/Cherry2/processed_images/PAT_1026_124_346.png',
 '/Users/theakaroline/Documents/GitHub/Cherry2/processed_images/PAT_1027_128_854.png',
 '/Users/theakaroline/Documents/GitHub/Cherry2/processed_images/PAT_1129_498_930.png',
 '/Users/theakaroline/Documents/GitHub/Cherry2/processed_images/PAT_1170_609_378.png',
 '/Users/theakaroline/Documents/GitHub/Cherry2/processed_images/PAT_1174_628_915.png',
 '/Users/theakaroline/Documents/GitHub/Cherry2/processed_images/PAT_1203_724_762.png',
 '/Users/theakaroline/Documents/GitHub/Cherry2/processed_images/PAT_1304_3507_308.png',
 '/Users/theakaroline/Documents/GitHub/Cherry2/processed_images/PAT_1306_1086_124.png',
 '/Users/theakaroline/Documents/GitHub/Cherry2/processed_images/PAT_1315_1110_317.png',
 '/Users/theakaroline/Documents/GitHub/Cherry2/processed_images/PAT_1414_1432_969.png',
 '/Users/theakaroline/Documents/GitHub/

In [98]:
def RGB2HEX(color):
     return "#{:02x}{:02x}{:02x}".format(int(color[0]), int(color[1]), int(color[2]))

#somethis hexcolor is easier, because it is not a tuple but a single line of code

In [172]:
def extract_red_scale(image):
    # Convert image to HSV color space
    #hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    im = cv2.imread(image)
    rgb_image = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    bw_image = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    red = [255, 0, 0]  # RGB
    diff = 50
    boundaries = [([red[0]-diff, red[1]-diff, red[2]-diff]),     ([red[0]+diff, red[1]+diff, red[2]+diff])]
    
    lower_red = np.array([1,0,0])
    upper_red = np.array([255,110,110])
    red_mask = cv2.inRange(rgb_image, lower_red, upper_red)

    output_red = rgb_image.copy()
    output_red[np.where(red_mask==0)] = 0
                         
    #plt.imshow(output_red)
    #plt.grid(None)
    
    #red_mask = cv2.inRange(image, lower_red, upper_red)
    # Compute the percentage of red pixels
    red_pixel_count = cv2.countNonZero(red_mask)
    total_pixels = cv2.countNonZero(bw_image)
    red_percentage = red_pixel_count / total_pixels
    #print(red_pixel_count)
    # Map red percentage to the desired scale
    
    if red_percentage < 0.05:
        red_scale = 0
    elif red_percentage < 0.2:
        red_scale = 1
    elif red_percentage < 0.4:
        red_scale = 2
    elif red_percentage < 0.6:
        red_scale = 3
    else:
        red_scale = 4
    #print((image.shape[0] * image.shape[1])-cv2.countNonZero(bw_image))
    #print(cv2.countNonZero(bw_image))
    return red_scale



1

In [173]:
def extract_yellow_scale(image):
    # Convert image to HSV color space
    #hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    im = cv2.imread(image)
    rgb_image = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    bw_image = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    yellow = [255, 255, 0]  # RGB
    diff = 50
    boundaries = [([yellow[0]-diff, yellow[1]-diff, yellow[2]-diff]),
                  ([yellow[0]+diff, yellow[1]+diff, yellow[2]+diff])]
    
    lower_yellow = np.array([220,120,120])
    upper_yellow = np.array([255,255,255])
    yellow_mask = cv2.inRange(rgb_image, lower_yellow, upper_yellow)

    output_yellow = rgb_image.copy()
    output_yellow[np.where(yellow_mask==0)] = 0

    
    #red_mask = cv2.inRange(image, lower_red, upper_red)
    # Compute the percentage of red pixels
    yellow_pixel_count = cv2.countNonZero(yellow_mask)
    total_pixels = cv2.countNonZero(bw_image)
    yellow_percentage = yellow_pixel_count / total_pixels

    # Map red percentage to the desired scale
    
    if yellow_percentage < 0.05:
        yellow_scale = 0
    elif yellow_percentage < 0.2:
        yellow_scale = 1
    elif yellow_percentage < 0.4:
        yellow_scale = 2
    elif yellow_percentage < 0.6:
        yellow_scale = 3
    else:
        yellow_scale = 4
    return yellow_scale



In [174]:
def extract_brown_scale(image):
    # Convert image to HSV color space
    #hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    im = cv2.imread(image)
    rgb_image = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    bw_image = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    brown = [255, 255, 0]  # RGB
    diff = 50
    boundaries = [([brown[0]-diff, brown[1]-diff, brown[2]-diff]),
                  ([brown[0]+diff, brown[1]+diff, brown[2]+diff])]
    
    lower_brown = np.array([50,0,0])
    upper_brown= np.array([150,200,100])
    brown_mask = cv2.inRange(rgb_image, lower_brown, upper_brown)

    output_brown = rgb_image.copy()
    output_brown[np.where(brown_mask==0)] = 0
                         
    #red_mask = cv2.inRange(image, lower_red, upper_red)
    # Compute the percentage of red pixels
    brown_pixel_count = cv2.countNonZero(brown_mask)
    total_pixels = cv2.countNonZero(bw_image)
    brown_percentage = brown_pixel_count / total_pixels

    # Map red percentage to the desired scale
    
    if brown_percentage < 0.05:
        brown_scale = 0
    elif brown_percentage < 0.2:
        brown_scale = 1
    elif brown_percentage < 0.4:
        brown_scale = 2
    elif brown_percentage < 0.6:
        brown_scale = 3
    else:
        brown_scale = 4
    return brown_scale


In [175]:
def extract_purple_scale(image):
    # Convert image to HSV color space
    #hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    im = cv2.imread(image)
    rgb_image = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    bw_image = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    purple = [255, 255, 0]  # RGB
    diff = 50
    boundaries = [([purple[0]-diff, purple[1]-diff, purple[2]-diff]),
                  ([purple[0]+diff, purple[1]+diff, purple[2]+diff])]
    
    lower_purple = np.array([50,99,0])
    upper_purple = np.array([130,100,255])
    purple_mask = cv2.inRange(rgb_image, lower_purple, upper_purple)

    output_purple = rgb_image.copy()
    output_purple[np.where(purple_mask==0)] = 0
    
    #red_mask = cv2.inRange(image, lower_red, upper_red)
    # Compute the percentage of red pixels
    purple_pixel_count = cv2.countNonZero(purple_mask)
    total_pixels = cv2.countNonZero(bw_image)
    purple_percentage = purple_pixel_count / total_pixels
    # Map red percentage to the desired scale
    
    if purple_percentage < 0.05:
        purple_scale = 0
    elif purple_percentage < 0.2:
        purple_scale = 1
    elif purple_percentage < 0.4:
        purple_scale = 2
    elif purple_percentage < 0.6:
        purple_scale = 3
    else:
        purple_scale = 4
    return purple_scale




In [180]:
def color_reader(image):
    im = cv2.imread(image)
    #     #we have to get the coordinates of every pixel in the image so
    xy_coords = np.flip(np.column_stack(np.where(im >= 0)), axis=1)
    
    #     This line calculates the coordinates of every pixel in the image where the pixel values are greater than or equal to 0. 
    #  Here's a step-by-step breakdown of what happens:

    # [Step 1] np.where(im >= 0) returns the indices where the condition im >= 0 is satisfied. It returns a tuple of arrays,
    #  one for each dimension of the image. 

    # [Step 2]np.column_stack combines the indices into columns. 
    # So, if the image has shape (height, width, channels), then the result will have shape (num_pixels, num_dimensions), 
    # where num_dimensions is the number of dimensions in the image (usually 3 for RGB images).

    # [Step 3]np.flip is used to reverse the order of columns in the resulting array.

    a_del = np.delete(xy_coords, 0, 1)
    a_del = a_del[::3][:, [0, 1]]
    
#    These lines manipulate the xy_coords array to prepare it for further processing:

    # [Step 1] np.delete(xy_coords, 0, 1) deletes the first column of xy_coords (which represents the color channel index, 
    # i.e., 0 for red, 1 for green, 2 for blue).

    # [Step 2]a_del[::3] selects every third row from a_del because each pixel has three color coordinates (RGB), 
    # so we only need one set of coordinates for each pixel.

    # [Step 3][:, [0, 1]] selects the first two columns of the resulting array, 
    # representing the x and y coordinates of each pixel. This is done using array indexing.


    image = Image.fromarray(im)
    rgb_image = image.convert('RGB')
    
    #  These lines convert the image from the OpenCV format (im) to an Image object (image).
    #  Then, the convert method is used to convert the image to the RGB color mode,
    #  creating a new Image object called rgb_image.

    rgb1 = [rgb_image.getpixel((int(i[0]),int(i[1]))) for i in a_del]
    
    #    This line retrieves the RGB values for each coordinate in a_del by calling the getpixel method of rgb_image. 
    # The getpixel method takes a tuple of coordinates (x, y) and returns the RGB values at that pixel location. 
    # The resulting RGB values are stored in the rgb1 list.

    dd = [RGB2HEX(i) for i in rgb1]
    ss = list(set(dd))
#   
    counter_colours =  Counter(dd)
#     #popping the black color
    counter_colours.pop('#000000')
     #print(counter_colours)
#     #plt.imshow(img)
    return counter_colours

#dont know if we should use this code or not

In [193]:
def hsv(image):    
    im = cv2.imread(image)
    #im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    new_arr_no_0 = im[np.where(im!=0)]
    image_copy = im.copy()
    non_black_pixels_mask = np.any(im != [0, 0, 0], axis=-1)  
    no_black = image_copy[non_black_pixels_mask]
    hsv_image = rgb2hsv(no_black)
    min_max = [np.amax(hsv_image[:,1]) - np.amin(hsv_image[:,1])]
    #print(np.amin(hsv_image[:,1]))
    return min_max

# this code is very cool actually


# A high difference in saturation means that there is a 
# large range of color intensity in the image, so some 
# parts of the image may have very intense colors while 
# others have muted colors.

# in our case, a picture with a high saturation (output close to 1) means
#  that there is a big variety in the color. 


In [213]:
def asymmetry_extraction(image):
    #01.read the image using the Matplotlib library 
    im = plt.imread(image)
    gray = cv2.cvtColor(np.float32(im), cv2.COLOR_BGR2GRAY)
    
    # Create a mask where non-black pixels are white
    mask = gray > 0
    mask=mask.astype(np.float32)
    #02.Crop the image : this is done by finding the indices of the non zero elements(pixels with non zero intensity values)
    #in order to do that,here we use np.nonezero from the numpy library 
    #center of the  shape is the center of the image
    #the boarders of the shape are the boaders of the image
    y_nonzero, x_nonzero = np.nonzero(mask)
    im = im[np.min(y_nonzero):np.max(y_nonzero), np.min(x_nonzero):np.max(x_nonzero)] #slice the image to include only the non zero regions

    #03.Cut the image in halves
    #04.Find the point of cutoff
    height, width = mask.shape
    cutoff_of_width = width//2
    cutoff_of_height  = height // 2
    
    #05.Cut the image vertically and horizontally in two
    vertical1_im = mask[:,:cutoff_of_width]
    vertical2_im = mask[:,cutoff_of_width:]
    
    horizontal1_im = mask[:cutoff_of_height,:]
    horizontal2_im = mask[cutoff_of_height:,:]

    #Simply put, in step 03, 04 and 05, cuts the cropped image into four sections: top, bottom , left and right.
    #This is done by dividing the width and height of the image by 2 and using the resulting values to slice the image horizontally and vertically


    #06.Flip one of the images both vertically and horizontally to create mirror images for each of the four sections.
    # This is done by NumPy array indexing with negative  step size to reverse the order of elements along the specified axis. 
    vertical_indexer = [slice(None)]*vertical2_im.ndim
    horizontal_indexer = [slice(None)]*horizontal2_im.ndim
    #Here, slice(None) creates a slice object that includes all elements along the specified axis, and imVertical2.ndim and imHorizontal2.ndim retrun the number of dimensions of the respective arrays.
    #create two lists of slice objects with the same lenght as the number of dimesnions of the corresponding arrays.





    vertical_indexer[1]= slice(None,None, -1)
    vertical_indexer[0]=slice(None, None,-1)
    #modify the slice objects in the second position  of each list to include all elements alng the corresponding axis in reverse order, using the -1 step size.
    #This flips the array along that axis effectively



    vertical2_im = vertical2_im[tuple(vertical_indexer)]
    horizontal2_im = horizontal2_im[tuple(horizontal_indexer)]
    #convert each list of slice objects to tuple and use it for NumPy indexing to obtain the flipped images.
 

    #07. if the images don't have the same shape, cut the biggest image
    #This can happen if the shape of the original shape was an odd number
    vertical2_im =vertical2_im[0:vertical1_im.shape[0], 0:vertical1_im.shape[1]]
    horizontal2_im = horizontal2_im[0:horizontal1_im.shape[0], 0:horizontal1_im.shape[1]]
    

    
    im_bwx_vertical = cv2.bitwise_xor(vertical1_im,vertical2_im)
    im_bwx_horizontal = cv2.bitwise_xor(horizontal1_im, horizontal2_im)
    
    
    vertical_area = np.sum(im_bwx_vertical==1)
    horizontal_area= np.sum(im_bwx_horizontal==1)
    mean_area = (vertical_area + horizontal_area) //2
    
    #np.sum is used to count the number of white pixels in each binary image im_bwx_vertical and im_bwx_horizontal. 
    #This count is performed using the logical expresion im_bwx_vertical == 1 and im_bwx_horizontal == 1 respectively. The expression returns a Boolean array that is True where the value of im_bwx_vertical or im_bwx_horizontal is equal to 1, and False elsewhere. The np.sum() function then counts the number of True values in the array and returns the total count of white pixels in each image.

    #Here, the asymmetry level is calculated as a percentage of the non zero pixels  in the overlapped image over the lesion area
    return(mean_area/np.sum(mask==1))*100
    #np.sum(im==1) is used to count the toatal number of non zero( while) pixels in the original image
    #(mean_area/np.sum(im==1))*100 calculates the asymmetry level of the  input image as a percentage.
    #the numerator is the average area of asymmetry  between two halves of the input image, in pixels.
    # The denominator us the total number of non zero pixels in the input image.
    #The result is  then multiplied by 100  to convert the values in to percentage.




In [202]:

def perimeter_area(image):
    im = plt.imread(image)
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    # Create a mask where non-black pixels are white
    mask = gray > 0
    # Set non-black pixels to white

    area = np.sum(mask) 
    struct_el = morphology.disk(3) 
    #cretaes  a disk shaped structuring element with a radius of 1, which wil be used for binary erosion.
    #Binary erosion is a morphological operation used in image processing to reduce the size of foreground objects(usually represented as white pixels) and eliminate isolated pixels or small components
    #The operation is performed by sliding a small binary structuring element (also known as kernel or mask) over the image and checking if all the pixels in the element match the corresponding pixels in the image.
    #If there is at least one mismatch, the pixel in the center of the structuring element is set to 0 (black), otherwise it is set to 1 (white).
    #As a result, the foreground pixels in the input image that are not completely covered by the structuring element are removed in the output image.
    # The size of the foreground objects is thus reduced, while their shape is preserved as much as possible.
    #The process is repeated iteratively until the foreground objects can no longer be further reduced, or until a desired level of erosion has been achieved.
    
 
    mask_eroded = morphology.binary_erosion(mask, struct_el)
   
    mask=mask.astype(np.float32)
    mask_eroded=mask_eroded.astype(np.float32)
    #erodes the binary mask using the strucutring element, resulting in a new binary mask with the same shape as the original mask but with the edges erroded.
    image_perimeter = mask - mask_eroded  #subtracts the eroded mask from the original mask to obtain an image that contains only the perimeter of the lesion.
    perimeter = np.sum(image_perimeter) #caculates the perimeter of the lesion by summing the values of all pixels in the perimeter image.
    return perimeter


In [204]:
def area(image):
    im = plt.imread(image)
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    # Create a mask where non-black pixels are white
    mask = gray > 0
    area = np.sum(mask) 
    return area

In [ ]:
#training and testing data : trying to split the data set into training data and  testing data
# import matplotlib.pyplot as plot
# from sklearn.datasets import load_files

# data_folder_path = '/Users/nuvanjanashashipraba/Desktop/Images'
# my_data = load_files(data_folder_path, categories =['folder1', 'folder2', 'folder3']) # loading the data from the folder path specified in step 3 and categorizing them into tthree folders

# X = my_data.data #assigning the image data to X
# Y = my_data.target #assigning the target labels to Y

# from sklearn.model_selection import train_test_split #importing the train_test_split function from the sklearn.model _slection library for splitting the data into trainin and testing sets.
# X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 2)
# #splitting the data into training and testing sets, with 20% of the data set aside for testing and setting the random state for reproducibility.
#The resulting four variables represent the training and testing data for both the image data.

#print("The dimensions of the features of training data ")
#print(X_train.shape)
#print("The dimensions of the features of test data ")
#print(X_test.shape)
#print("The dimensions of the target values of training data ")
#print(Y_train.shape)
#print("The dimensions of the target values of test data ")
#print(Y_test.shape)

In [30]:
def knn_prediction(k, train, classes, test): #classes : the class labels of the training data.
    neighb = KNeighborsClassifier(n_neighbors= k)
    neighb.fit(train, classes.ravel())
    classes_pred = neighb.predict(test)
    return classes_pred

In [31]:
def accuracy_test(k,train,classes,test,classes_test):
    performance = []
    perform1 =[]
    #the 'performance' list stores the predicted lables for each value of k
    #the 'perform1' list stores the accuracy score for each value of k

    for i in range(1,k):
        a = make_knn_prediction(i,train,classes,test)
        performance.append(a)
        for j in performance:
            b = accuracy_score(classes_test,j) #'accuracy_score' is function from scikit learn 
        perform1.append(b)
    fig, axes = plt.subplots()
    axes.plot(perform1)
    plt.title("Classification Accuracy of KNN for Different Values of k")
    plt.ylabel("Test Accuracy")
    plt.xlabel("Value of k");
    plt.grid(color='grey', linestyle='--', linewidth=0.5)
    return np.mean(perform1)

### Splittting the data set for the color

In [236]:
red_values = list(map(extract_red_scale, path_to_all_images))
yellow_values = list(map(extract_yellow_scale, path_to_all_images))
brown_values = list(map(extract_brown_scale, path_to_all_images))
purple_values = list(map(extract_purple_scale, path_to_all_images))

to_be_cancerous_or_not_to_be['red'] = np.array(red_values)
to_be_cancerous_or_not_to_be['yellow'] = np.array(yellow_values)
to_be_cancerous_or_not_to_be['brown'] = np.array(brown_values)
to_be_cancerous_or_not_to_be['purple'] = np.array(purple_values)
to_be_cancerous_or_not_to_be
# print(df['new_col'] = mylist)

,Cancerous,red,yellow,brown,purple
ID,,,,,
PAT_1022_115_132,0,1,0,0,0
PAT_1026_124_346,0,0,0,0,0
PAT_1027_128_854,0,0,0,0,1
PAT_1129_498_930,0,2,0,0,0
PAT_1170_609_378,0,0,0,0,1
...,...,...,...,...,...
PAT_944_1795_666,1,0,0,0,0
PAT_952_1807_58,1,0,0,0,0
PAT_97_151_503,1,0,0,0,0


In [277]:
# colors_values = list(map(color_reader, path_to_all_images))
# to_be_cancerous_or_not_to_be['colors'] = np.array(colors_values)
# to_be_cancerous_or_not_to_be

In [237]:

    
hsv_values = list(map(hsv, path_to_all_images))
asymmetry_values = list(map(asymmetry_extraction, path_to_all_images))
perimeter_values = list(map(perimeter_area, path_to_all_images))
area_values = list(map(area, path_to_all_images))


to_be_cancerous_or_not_to_be['Perimeter'] = np.array(perimeter_values)
to_be_cancerous_or_not_to_be['Area'] = np.array(area_values)
to_be_cancerous_or_not_to_be['Asymmetry'] = np.array(asymmetry_values)
to_be_cancerous_or_not_to_be['hsv'] = np.array(hsv_values)


In [240]:
to_be_cancerous_or_not_to_be

,Cancerous,red,yellow,brown,purple,Perimeter,Area,Asymmetry,hsv
ID,,,,,,,,,
PAT_1022_115_132,0,1,0,0,0,3291.0,47497,46.670316,0.918919
PAT_1026_124_346,0,0,0,0,0,3591.0,86745,42.428958,0.488057
PAT_1027_128_854,0,0,0,0,1,1077.0,10017,85.554557,0.236839
PAT_1129_498_930,0,2,0,0,0,2031.0,13054,99.593994,0.835294
PAT_1170_609_378,0,0,0,0,1,4334.0,148387,62.557367,0.663069
...,...,...,...,...,...,...,...,...,...
PAT_944_1795_666,1,0,0,0,0,16164.0,106635,64.147794,1.000000
PAT_952_1807_58,1,0,0,0,0,8896.0,50535,69.377659,1.000000
PAT_97_151_503,1,0,0,0,0,10607.0,219418,48.223482,1.000000


In [274]:
#X should be everything but resutlt, y shoud be result
Y= to_be_cancerous_or_not_to_be.iloc[:,:1]
X = to_be_cancerous_or_not_to_be.iloc[:, 1:]
y = Y.squeeze()
y

ID
PAT_1022_115_132    0
PAT_1026_124_346    0
PAT_1027_128_854    0
PAT_1129_498_930    0
PAT_1170_609_378    0
                   ..
PAT_944_1795_666    1
PAT_952_1807_58     1
PAT_97_151_503      1
PAT_987_1859_859    0
PAT_990_1860_636    1
Name: Cancerous, Length: 98, dtype: int64

In [62]:
#checking the path for the folder
import os

# Get the path to the images folder
folder_name = "processed_images"
folder_path = os.path.join(os.getcwd(), folder_name)

# Print the path
print(folder_path)


/Users/nuvanjanashashipraba/Documents/GitHub/cherry2/processed_images


In [243]:
#training and testing data : trying to split the data set into training data and  testing data
import matplotlib.pyplot as plot


data_folder_path = '/Users/theakaroline/Documents/GitHub/cherry2/processed_images'


img_files = list(os.listdir(image_folder_path))


train_files, test_files = train_test_split(to_be_cancerous_or_not_to_be, test_size=0.2, random_state = 2)

train_files_S, test_files_S = train_test_split(to_be_cancerous_or_not_to_be, test_size=0.2, random_state = 2, stratify=to_be_cancerous_or_not_to_be[["Cancerous"]])
#Print number of images in each set
print('Number of training images:', len(train_files))
print('Number of testing images:', len(test_files))

print('Number of training images with stratify:', len(train_files_S))
print('Number of testing images with stratify:', len(test_files_S))

Number of training images: 78
Number of testing images: 20
Number of training images with stratify: 78
Number of testing images with stratify: 20


In [275]:
k = 5
kf = KFold(n_splits=k, random_state=None)
model = LogisticRegression(solver= 'liblinear')
 
acc_score = []
 
for train_index , test_index in kf.split(X):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]
     
    model.fit(X_train,y_train)
    pred_values = model.predict(X_test)
     
    acc = accuracy_score(pred_values , y_test)
    acc_score.append(acc)
     
avg_acc_score = sum(acc_score)/k

print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))

accuracy of each fold - [0.0, 0.6, 0.2, 0.3157894736842105, 0.3684210526315789]
Avg accuracy : 0.2968421052631579


In [276]:

from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

from sklearn.model_selection import cross_val_score
clf = svm.SVC(kernel='linear', C=1, random_state=42)
scores = cross_val_score(clf, X, y, cv=5)
scores

array([0.35      , 0.7       , 0.5       , 0.15789474, 0.89473684])

<class 'pandas.core.series.Series'>


In [56]:
# color_data = my_data.to_numpy() # what is the df_features_extra in our case? what is the "Nb_of-colours" in our case?
# #df_features = the name of the csv or the varibale name that you used to read the csv file or the data set
# #because our data set is not a csv  file, can we just write: name of the dataset.to_numpy()

# color_train, color_test, clas_train, clas_test = train_test_split(color_data, clas, test_size = 0.6, random_state= 4)

# color_train = color_train. reshape(-1, 1)
# clas_train = clas_train.reshape (-1, 1)
# color_test = color_test.reshape( -1, 1)

In [ ]:
print(knn_prediction(7, color_train, clas_train, color_test))
print(clas_test)

In [ ]:
accuracy_test(380,color_train, clas_train, color_test, clas_test)

### Splitting the data set for the Saturation 

In [ ]:
sat_data = df_features_extra["sat_value"].to_numpy()
sat_train, sat_test, clas_train, clas_test = train_test_split(sat_data,clas,test_size=0.6, random_state=4)

sat_train = sat_train,reshape(-1,1)
clas_train = clas_train.reshape(-1,1)
sat_test = sat_test.reshape(-1,1)


In [ ]:
print(knn_prediction(7,sat_train, clas_train,sat_test))
print(clas_test)

In [ ]:
accuracy_test(150, sat_train,clas_train,sat_test,clas_test)


### Splitting the data set for the Asymmetry

In [ ]:
#Splittng the data set for the Asymmetry
aymmetry_data =df_features_extra ["Aymmetry"].to_numpy() #but what is the df_features_extra in our case? # what is the "Asymmetry" in our case.
#df_features = the name of the csv or the varibale name that you used to read the csv file or the data set
#because our data set is not a csv  file, can we just write: name of the dataset.to_numpy()
asymmetry_train = asymmetry_test, clas_train, clas_test = train_test_split(asymmetry_data, clas, test_size = 0.6, random_state= 4)
asymmetry_train = asymmetry_train.reshape(-1,1)
clas_train = clas_train.reshape(-1, 1)
asymmetry_test = asymmetry_test.reshape(-1,1)

In [ ]:
print(knn_prediction(7,asymmetry_train, clas_train,asymmetry_test))
print(clas_test)


In [ ]:
accuracy_test(800,asymmetry_train,clas_train,asymmetry_test.clas_test)
